# HW-1

In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
PATH_TO_DATA = './data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)
data.head(3)

,content,keywords,summary,title,url
0,В среду состоялось отложенное заседание Совета...,"[школа, образовательные стандарты, литература,...","Глава Минобрнауки считает, что в нездоровом аж...","Ольга Васильева обещала ""НГ"" не перегружать шк...",https://amp.ng.ru/?p=http://www.ng.ru/educatio...
1,"Хорошо, когда красота в глазах смотрящего живе...","[красота, законы]",О живительной пользе укорота при выборе между ...,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/20...
2,Когда-то Леонид Юзефович написал книгу о монго...,"[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisima...


In [3]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        if (tp+fp) == 0:prec = 0
        else:prec = tp / (tp + fp)
        if (tp+fn) == 0: rec = 0
        else: rec = tp / (tp + fn)
        if (prec+rec) == 0: f1 = 0
        else: f1 = (2*(prec*rec))/(prec+rec)   
        jac = tp / union
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [4]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


In [4]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'

In [5]:
import re

## "Ухудшение" 1. Что-то вроде Rake

Выбираем последовательности из существительных и прилагательных так, что либо (adj)* (noun)* либо (noun)* , так как термины и ключевые слова утсроены так. Берем степень, частоту слова в целом, считаем коэффициент, для сочетания просто все складываем по составляющим. Так как наши ключевые слвоа все 1-2 слова длиной, берем только окончания таких сочетаний (последние 2 слова), иначе попадают в топ длинные последовательности.

In [6]:
from nltk.tokenize import wordpunct_tokenize

import numpy as np

from itertools import combinations

In [8]:
def normalize2(text):
    
    words = [word.strip(punct) for word in wordpunct_tokenize(text.lower())]
    words = [morph.parse(word)[0] if word and word not in stops else morph.parse('и')[0] for word in words ]
    adj = True
    add = False
    words2 = []
    for word in words:
        if word.tag.POS == 'NOUN':
            if add:
                words2[-1].append(word.normal_form)
                adj = False
            else:
                words2.append([word.normal_form])
                add = True
                adj = False
        elif word.tag.POS == 'ADJF':
            if adj and add:
                    words2[-1].append(word.normal_form)
            else:
                    words2.append([word.normal_form])                
            adj = True
            add = True
        else:
            adj = True
            add = False
    return words2

In [10]:
data['content_norm2'] = data['content'].apply(normalize2)

In [11]:
def get_scores(text):
    a = set()
    for i in text:
        a.update(i)
    a = list(a)
    matrix = np.zeros((len(a), len(a)))
    for i in text:
        for j in combinations(i[-3:], 2):
            x = a.index(j[0])
            y = a.index(j[1])
            matrix[x, y] += 1
            matrix[y, x] += 1
        for j in i[-2:]:
            x = a.index(j)
            matrix[x,x] += 1
    scores = np.zeros((len(a), 3))
    norm = matrix.max()
    for key, i in enumerate(a):
        scores[key, 0] = sum(matrix[key,])
        scores[key, 1] = matrix[key,key]
        scores[key, 2] = scores[key, 0]
    result = {}
    for i in text:
        k = i[-2:]
        score = 0
        for j in k:
            key = a.index(j)
            score += scores[key, 2]#*(scores[key, 1]/norm)
        if len(k) == 2:
            score = score * matrix[a.index(k[0]), a.index(k[1])] / norm
        elif len(k) == 1:
            score = score * matrix[a.index(k[0]), a.index(k[0])] / norm
        if ' '.join(k) in result:
            result[' '.join(k)] += score
        else:
            result[' '.join(k)] = score
    return result

In [12]:
d = get_scores(data.iloc[0]['content_norm2'])
for i in sorted(d, key=d.get, reverse=True)[:10]:
    print (i, d[i])

исторический источник 153.0
ольга васильев 117.0
который 64.0
стандарт 19.0
образовательный стандарт 18.5
предметный результат 15.0
источник информация 15.0
образовательный программа 13.0
образовательный пространство 12.0
редакция фгоса 11.0


In [13]:
def get_kw_r(text):
    d = get_scores(text)
    return list(sorted(d, key=d.get, reverse=True))[:5]

In [14]:
keyowords = data['content_norm2'].apply(get_kw_r)

In [15]:
evaluate(data['keywords'], keyowords)

Precision -  0.14
Recall -  0.14
F1 -  0.13
Jaccard -  0.08


Работает не очень по сравнению с tfidf, однако эта штука не зависит от общей выборки, полностью внутри текста - это иногда очень полезно.

## "Ухудшение" 2. Тот же псевдорэйк + tfidf

Прошлое + tfidf. Коэффициент умножается на 1 + tfidf, чтобы tfidf был как бы повышающим коэффициентом.

In [16]:
def double_join(text):
    text = [' '.join(i) for i in text]
    return ' '.join(text)

In [17]:
data['content_norm_str'] = data['content_norm2'].apply(double_join)

In [18]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [19]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [20]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [21]:
word2id = {id2word[i]:i for i in id2word}

In [22]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [23]:
word2id['образовательный стандарт']

13828

In [24]:
texts_vectors[0]

<1x27562 sparse matrix of type '<class 'numpy.float64'>'
	with 271 stored elements in Compressed Sparse Row format>

In [25]:
def get_scores(text):
    a = set()
    for i in text:
        a.update(i)
    a = list(a)
    matrix = np.zeros((len(a), len(a)))
    for i in text:
        for j in combinations(i[-3:], 2):
            x = a.index(j[0])
            y = a.index(j[1])
            matrix[x, y] += 1
            matrix[y, x] += 1
        for j in i[-2:]:
            x = a.index(j)
            matrix[x,x] += 1
    scores = np.zeros((len(a), 3))
    norm = matrix.max()
    for key, i in enumerate(a):
        scores[key, 0] = sum(matrix[key,])
        scores[key, 1] = matrix[key,key]
        scores[key, 2] = scores[key, 0]
    result = {}
    for i in text:
        k = i[-2:]
        score = 0
        for j in k:
            key = a.index(j)
            score += scores[key, 2]#*(scores[key, 1]/norm)
        #if len(k) == 2:
        #    score = score * matrix[a.index(k[0]), a.index(k[1])] / norm
        #elif len(k) == 1:
        #    score = score * matrix[a.index(k[0]), a.index(k[0])] / norm
        if ' '.join(k) in result:
            result[' '.join(k)] += score
        else:
            result[' '.join(k)] = score
    return result

In [27]:
keywords = []
for key, value in enumerate(texts_vectors):
    d = get_scores(data.iloc[key]['content_norm2'])
    for i in d:
        if i in word2id:
            d[i] = d[i]*(1+value[0][0, word2id[i]])
        else:
            d[i] = 0
    keywords.append(list(sorted(d, key=d.get, reverse=True))[:5])
    #print (list(sorted(d, key=d.get, reverse=True))[:5])

In [28]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


## "Ухудшение" 3. То же самое, но другая метрика

В этом случае наоборот, rake коэффициент является повышающим для tfidf. (1+log(rake)) * tfidf

In [29]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ('NOUN', 'ADJF')]

    return words
data['content_norm'] = data['content'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [30]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


In [31]:
def normalize2(text):
    
    words = [word.strip(punct) for word in wordpunct_tokenize(text.lower())]
    words = [morph.parse(word)[0] if word and word not in stops else morph.parse('и')[0] for word in words ]
    adj = True
    add = False
    words2 = []
    for word in words:
        if word.tag.POS == 'NOUN':
            if add:
                words2[-1].append(word.normal_form)
                adj = False
            else:
                words2.append([word.normal_form])
                add = True
                adj = False
        elif word.tag.POS == 'ADJF':
            if adj and add:
                    words2[-1].append(word.normal_form)
            else:
                    words2.append([word.normal_form])                
            adj = True
            add = True
        else:
            adj = True
            add = False
    return words2
data['content_norm2'] = data['content'].apply(normalize2)

In [32]:
import math

In [33]:
keywords = []
word2id = {id2word[i]:i for i in id2word}
for key, value in enumerate(texts_vectors):
    d = get_scores(data.iloc[key]['content_norm2'])
    for i in d:
        if i in word2id:
            d[i] = (1+math.log(d[i]))*value[0][0, word2id[i]]
        else:
            d[i] = 0
    keywords.append(list(sorted(d, key=d.get, reverse=True))[:5])
evaluate(data['keywords'], keywords)

Precision -  0.17
Recall -  0.17
F1 -  0.16
Jaccard -  0.1


# -----------------------------------------------------------------------------

## Улучшение 1. Препроцессинг

Так как в ключевых словах встречаются не только существительные, но и прилагательные, нужно добавить и их. Плюс ключевые слова часто стоят во множественном числе и у прилагательных какой-то род, поэтому наши правильные ключевые слова могут просто не подходить по этим параметрам. Поэтому мы сохраняем род и число пирлагательных такими, какие они есть, просто переводим их и существительные в именительный падеж.

In [34]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}).word for word in words if word.tag.POS in ('NOUN', 'ADJF')]

    return words

In [35]:
data['content_norm'] = data['content'].apply(normalize)

In [36]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [37]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [38]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [39]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 
evaluate(data['keywords'], keywords)

Precision -  0.2
Recall -  0.2
F1 -  0.19
Jaccard -  0.11


## Улучшение 2. Постпроцессинг

Ок, мы можем там терять термины, которые то во мн.ч, то в единственном. Можно считать все за одно, но запоминать, какие параметры (число, род, генитив) были часто у них в тексте. Найти ключевые в нормальной форме, а потом просклонять в нужную форму. Прилагательные: по роду и числу, существительные: по числу. Если существительное в основном в генитиве (типа совет федерации), то его в генитив. 


Сначала только для существительных, потом для существительных и прилагательных.

In [40]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

data['content_norm'] = data['content'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [41]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])

In [42]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [9]:
def get_noun_grammar(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words2 = {}
    for word in words:
        if word.tag.POS == 'NOUN':
            lemma = word.normal_form
            if lemma not in words2:
                words2[lemma]={'sing':0, 'plur':0, 'gent':0, 'total':0}
            words2[lemma][word.tag.number] += 1
            if word.tag.case == 'gent':
                words2[lemma]['gent'] += 1
            words2[lemma]['total']+=1
    return words2

In [10]:
def inflect_item(keyword, grammar):
    keyword = [morph.parse(i)[0] for i in keyword.split()]
    #print (keyword)
    result = []
    if len(keyword) == 2:
        if keyword[0].tag.POS == 'NOUN' and keyword[1].tag.POS == 'NOUN':
            try:
                if grammar[keyword[0].normal_form]['plur'] > grammar[keyword[0].normal_form]['sing']:
                    result.append(keyword[0].inflect({'plur'}).word)
                else:
                    result.append(keyword[0].normal_form)
            except:
                result.append(keyword[0].normal_form)
            
            try:
                if grammar[keyword[1].normal_form]['gent'] > 0.4*grammar[keyword[1].normal_form]['total']:
                    if grammar[keyword[1].normal_form]['plur'] > grammar[keyword[1].normal_form]['sing']:
                        result.append(keyword[1].inflect({'gent', 'plur'}).word)
                    else: result.append(keyword[1].inflect({'gent'}).word)
            except:
                result.append(keyword[1].normal_form)
        
        elif keyword[0].tag.POS == 'ADJF' and keyword[1].tag.POS == 'NOUN':
            gender = keyword[1].tag.gender
            try:
                
                if grammar[keyword[1].normal_form]['plur'] > grammar[keyword[1].normal_form]['sing']:
                    number = 'plur'
                    result.append(keyword[0].inflect({number}).word)
                    result.append(keyword[1].inflect({number}).word)
                else:
                    number = 'sing'
                    result.append(keyword[0].inflect({gender}).word)
                    result.append(keyword[1].inflect({number}).word)
            except:
                result.append(keyword[0].inflect({gender}).word)
                
                result.append(keyword[1].normal_form)
        else:
            result = [i.normal_form for i in keyword]
    elif len(keyword) == 1:
        if keyword[0].tag.POS == 'NOUN':
            if keyword[0].normal_form in grammar:
                if grammar[keyword[0].normal_form]['plur'] > grammar[keyword[0].normal_form]['sing']:
                    number = 'plur'
                else: number = 'sing'
                try:
                    result.append(keyword[0].inflect({number}).word)
                except:
                    result.append(keyword[0].normal_form)
            
            else:
                result = [i.normal_form for i in keyword]
        else:
            result = [i.normal_form for i in keyword]
    else:
        result = [i.normal_form for i in keyword]
    return ' '.join(result)

In [45]:
kw = []
for key, value in enumerate(keywords):
    kw.append([])
    #print (keywords[key])
    grammar = get_noun_grammar(data.iloc[key]['content'])
    #print (grammar)
    for item in value:
        kw[-1].append(inflect_item(item, grammar))

In [46]:
evaluate(data['keywords'], kw)

Precision -  0.14
Recall -  0.26
F1 -  0.18
Jaccard -  0.1


In [48]:
kw = []
for key, value in enumerate(keywords):
    kw.append([])
    #print (keywords[key])
    grammar = get_noun_grammar(data.iloc[key]['content'])
    #print (grammar)
    for item in value[:5]:
        kw[-1].append(inflect_item(item, grammar))
evaluate(data['keywords'], kw)

Precision -  0.2
Recall -  0.19
F1 -  0.19
Jaccard -  0.12


Это уже лучше бэйзлайна. Добавим прилагательные. Оставим топ-5 ключевых.

In [7]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ('NOUN','ADJF')]

    return words

data['content_norm'] = data['content'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [8]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [11]:
kw = []
for key, value in enumerate(keywords):
    kw.append([])
    #print (keywords[key])
    grammar = get_noun_grammar(data.iloc[key]['content'])
    #print (grammar)
    for item in value:
        kw[-1].append(inflect_item(item, grammar))
evaluate(data['keywords'], kw)

Precision -  0.19
Recall -  0.18
F1 -  0.18
Jaccard -  0.11


Прилагательные на так часто встречаются, при этом ухудшают результат в среднем

## Улучшение 3: NER

Имена встречаются среди ключевых слов, хочется их достать как-то по-нормальному и учесть их в разных формах (только фамилия, фамилия и имя). Для этого используем natasha.

1. посмотреть только на имена (плохо, не везде они есть)
2. посмотреть с базовой предобработкой (что если не нагроиождать сложную грамматическую информацию, а просто приклеить имена, насколько это повлияет на результат)

In [13]:
from natasha import PersonExtractor
extractor_per = PersonExtractor()

In [15]:
def get_names(text):
    matches = extractor_per(text)
    spans = [_.span for _ in matches]
    facts = [_.fact.as_json for _ in matches]
    result = []
    for i in facts:
        if 'last' in i['name']:
            s = ' '.join(j for j in i['name'].values())
            if len(s) > 3:
                result.append(s)
            s = i['name']['last']
            if len(s) > 3:
                result.append(s)
    result = Counter(result)
    result = [i for i in sorted(result, key=result.get, reverse=True) if result[i] > 2]
    return result

In [16]:
names = [get_names(i) for i in data['content'].values]

In [17]:
evaluate(data['keywords'], names)

Precision -  0.09
Recall -  0.04
F1 -  0.04
Jaccard -  0.03


In [18]:
from Levenshtein import distance

In [19]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ('NOUN',)]

    return words

data['content_norm'] = data['content'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [20]:
new = []
for key, value in enumerate(names):
    k, ind = 0, 0
    r = ['']+[i for i in value]
    while len(r)<10 and ind != len(keywords[key]):
        w = min(distance(j, keywords[key][ind]) for j in r)
        if w > 2:
            r.append(keywords[key][ind])
            k += 1
        ind += 1
    new.append(r[1:])

In [21]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [22]:
evaluate(data['keywords'], new)

Precision -  0.14
Recall -  0.24
F1 -  0.17
Jaccard -  0.1
